# What is the effectiveness of the pre-trained model in commit message generation?

In [1]:
import pandas as pd
import itertools, os,json

model_name = "CodeBERT"
dataset_name = "MCMD"
lan_list = ["java", "python", "javascript", "cpp", "csharp"]
metric_list = ['B-Moses', 'B-Norm', 'B-CC', 'Rouge-1', 'Rouge-2', 'Rouge-L', 'Meteor']
display_lan_list = lan_list
lan_list.sort()

def comb_finetune(test_pl, lan_n):
    results = pd.DataFrame(columns=metric_list, index=list(itertools.combinations(lan_list, lan_n)))
    for comb in itertools.combinations(lan_list, lan_n):
        finetune_pl_comb = "_".join(comb)
        ref_path = "../experiment_results/model_CodeBERT/data_MCMD/{}/ref.msg".format(test_pl)
        if lan_n == 1 and test_pl==finetune_pl_comb:
            gen_path = "../experiment_results/model_CodeBERT/data_MCMD/{}/gen.msg".format(test_pl, finetune_pl_comb)
        else:
            gen_path = "../experiment_results/model_CodeBERT/data_MCMD/{}/finetune_{}/gen.msg".format(test_pl, finetune_pl_comb)
        BM_evaluate_cmd = "../metrics/B-Moses.perl {} < {}".format(ref_path, gen_path)
        results['B-Moses'][comb] = float(os.popen(BM_evaluate_cmd).read().split("=")[1].split(",")[0].strip())
        BN_evaluate_cmd = "python ../metrics/B-Norm.py {} < {}".format(ref_path, gen_path)
        results['B-Norm'][comb] = float(os.popen(BN_evaluate_cmd).read())
        BC_evaluate_cmd = "python ../metrics/B-CC.py --ref_path {} --gen_path {}".format(ref_path, gen_path)
        results['B-CC'][comb] = float(os.popen(BC_evaluate_cmd).read().strip())
        Rouge_evaluate_cmd = "python ../metrics/Rouge.py --ref_path {} --gen_path {}".format(ref_path, gen_path)
        rouge_str = os.popen(Rouge_evaluate_cmd).read().replace("'", "\"")
        rouge_dict = json.loads(rouge_str)
        results['Rouge-1'][comb], results['Rouge-2'][comb], results['Rouge-L'][comb] = rouge_dict['ROUGE-1'], rouge_dict['ROUGE-2'], rouge_dict['ROUGE-L']
        Meteor_evaluate_cmd = "python ../metrics/Meteor.py --ref_path {} --gen_path {}".format(ref_path, gen_path)
        results['Meteor'][comb] = float(os.popen(Meteor_evaluate_cmd).read().strip())
    pd.options.display.float_format = '{:.2f}'.format
    return results

## Test PL: Java

### MCMD<sub>1-PL</sub>

In [2]:
comb_finetune("java",1)

,B-Moses,B-Norm,B-CC,Rouge-1,Rouge-2,Rouge-L,Meteor
"(cpp,)",2.04,7.08,3.26,9.23,3.07,9.01,6.98
"(csharp,)",2.21,6.82,3.61,8.83,3.08,8.65,7.70
"(java,)",10.79,18.98,13.02,22.74,11.95,22.34,20.99
"(javascript,)",1.93,6.92,3.14,9.72,2.82,9.43,7.47
"(python,)",2.32,7.06,3.39,10.31,3.49,10.02,7.79


### MCMD<sub>2-PL</sub>

In [5]:
comb_finetune("java",2)

,B-Moses,B-Norm,B-CC,Rouge-1,Rouge-2,Rouge-L,Meteor
"(cpp, csharp)",3.08,7.64,4.05,10.13,3.85,9.87,8.10
"(cpp, java)",10.54,18.56,12.76,21.97,11.70,21.60,20.35
"(cpp, javascript)",2.41,7.05,3.52,8.71,3.16,8.45,6.94
"(cpp, python)",2.50,7.39,3.70,9.59,3.65,9.30,7.41
"(csharp, java)",10.78,18.76,12.97,22.27,11.84,21.89,20.71
"(csharp, javascript)",2.42,7.28,3.45,10.33,3.37,10.01,8.15
"(csharp, python)",2.46,7.66,3.78,10.50,3.42,10.18,8.53
"(java, javascript)",10.75,18.86,13.00,22.41,11.93,22.01,20.82
"(java, python)",10.81,18.89,13.04,22.47,11.94,22.06,20.88
"(javascript, python)",2.46,7.90,3.75,11.40,3.64,11.00,8.79


### MCMD<sub>3-PL</sub>

In [4]:
comb_finetune("java",3)

,B-Moses,B-Norm,B-CC,Rouge-1,Rouge-2,Rouge-L,Meteor
"(cpp, csharp, java)",10.76,18.58,12.87,22.09,11.72,21.70,20.52
"(cpp, csharp, javascript)",2.66,7.44,3.76,9.41,3.58,9.12,7.35
"(cpp, csharp, python)",2.89,7.83,4.10,10.09,3.84,9.77,7.92
"(cpp, java, javascript)",10.94,19.00,13.18,22.72,12.11,22.29,21.06
"(cpp, java, python)",10.81,18.86,13.05,22.53,12.05,22.12,20.83
"(cpp, javascript, python)",2.55,7.40,3.75,8.95,3.35,8.66,7.20
"(csharp, java, javascript)",10.92,18.91,13.11,22.57,12.05,22.16,20.91
"(csharp, java, python)",10.70,18.67,12.84,22.25,11.83,21.87,20.54
"(csharp, javascript, python)",2.64,8.33,4.04,11.41,3.69,11.02,9.32
"(java, javascript, python)",10.83,18.77,13.02,22.47,11.98,22.05,20.82


### MCMD<sub>4-PL</sub>

In [5]:
comb_finetune("java",4)

,B-Moses,B-Norm,B-CC,Rouge-1,Rouge-2,Rouge-L,Meteor
"(cpp, csharp, java, javascript)",10.88,18.58,12.97,22.12,11.85,21.71,20.62
"(cpp, csharp, java, python)",8.57,15.80,10.54,19.23,10.21,18.83,16.58
"(cpp, csharp, javascript, python)",2.98,8.29,4.17,11.29,3.94,10.91,9.01
"(cpp, java, javascript, python)",10.81,18.88,12.99,22.60,11.89,22.15,20.86
"(csharp, java, javascript, python)",10.90,18.93,13.07,22.72,12.09,22.28,20.93


### MCMD<sub>5-PL</sub>

In [6]:
comb_finetune("java",5)

,B-Moses,B-Norm,B-CC,Rouge-1,Rouge-2,Rouge-L,Meteor
"(cpp, csharp, java, javascript, python)",11.03,19.10,13.24,22.89,12.16,22.43,21.21


## Test PL: Python (Py)

### MCMD<sub>1-PL</sub>

In [11]:
comb_finetune("python",1)

,B-Moses,B-Norm,B-CC,Rouge-1,Rouge-2,Rouge-L,Meteor
"(cpp,)",2.75,9.45,4.09,13.59,5.30,13.23,9.03
"(csharp,)",1.73,7.65,3.04,10.58,3.70,10.34,7.38
"(java,)",2.10,7.74,3.55,10.70,3.48,10.38,7.96
"(javascript,)",2.02,8.12,3.30,11.89,3.98,11.57,7.91
"(python,)",10.55,19.39,12.97,25.28,13.48,24.63,22.69


### MCMD<sub>2-PL</sub>

In [18]:
comb_finetune("python",2)

,B-Moses,B-Norm,B-CC,Rouge-1,Rouge-2,Rouge-L,Meteor
"(cpp, csharp)",2.97,9.63,4.31,13.59,5.45,13.24,9.37
"(cpp, java)",2.90,9.20,4.19,13.51,5.52,13.14,9.18
"(cpp, javascript)",2.95,9.29,4.15,13.03,5.25,12.65,8.47
"(cpp, python)",9.56,17.56,11.60,23.30,12.55,22.69,19.59
"(csharp, java)",2.27,8.44,3.65,12.25,4.50,11.88,8.37
"(csharp, javascript)",2.44,8.70,3.78,12.90,4.84,12.50,8.63
"(csharp, python)",10.84,19.33,13.11,25.01,13.48,24.35,22.70
"(java, javascript)",2.56,9.27,3.94,13.71,4.96,13.27,9.05
"(java, python)",10.66,19.28,12.98,25.02,13.45,24.36,22.64
"(javascript, python)",10.87,19.47,13.16,25.33,13.53,24.64,22.80


### MCMD<sub>3-PL</sub>

In [19]:
comb_finetune("python",3)

,B-Moses,B-Norm,B-CC,Rouge-1,Rouge-2,Rouge-L,Meteor
"(cpp, csharp, java)",2.67,9.02,3.97,13.23,5.08,12.84,8.90
"(cpp, csharp, javascript)",3.15,9.81,4.40,13.54,5.46,13.13,8.81
"(cpp, csharp, python)",8.53,16.39,10.34,21.84,11.29,21.14,18.13
"(cpp, java, javascript)",2.95,9.69,4.32,14.37,5.71,13.90,9.65
"(cpp, java, python)",10.86,19.26,13.14,25.16,13.63,24.48,22.70
"(cpp, javascript, python)",6.62,14.53,8.24,19.51,9.64,18.90,14.62
"(csharp, java, javascript)",2.53,9.07,3.91,13.64,5.15,13.20,8.90
"(csharp, java, python)",10.74,19.23,12.98,25.01,13.46,24.36,22.49
"(csharp, javascript, python)",11.00,19.56,13.26,25.42,13.69,24.70,22.92
"(java, javascript, python)",10.84,19.34,13.15,25.20,13.61,24.52,22.71


### MCMD<sub>4-PL</sub>

In [20]:
comb_finetune("python",4)

,B-Moses,B-Norm,B-CC,Rouge-1,Rouge-2,Rouge-L,Meteor
"(cpp, csharp, java, javascript)",2.89,9.53,4.25,14.02,5.48,13.60,9.47
"(cpp, csharp, java, python)",8.34,16.42,10.22,21.82,11.17,21.09,18.10
"(cpp, csharp, javascript, python)",10.90,19.43,13.13,25.33,13.56,24.62,22.79
"(cpp, java, javascript, python)",10.97,19.38,13.23,25.25,13.67,24.54,22.76
"(csharp, java, javascript, python)",11.02,19.41,13.28,25.33,13.75,24.65,22.96


### MCMD<sub>5-PL</sub>

In [21]:
comb_finetune("python",5)

,B-Moses,B-Norm,B-CC,Rouge-1,Rouge-2,Rouge-L,Meteor
"(cpp, csharp, java, javascript, python)",11.10,19.58,13.35,25.51,13.73,24.79,22.98


## Test PL: JavaScript (Js)

### MCMD<sub>1-PL</sub>

In [12]:
comb_finetune("javascript",1)

,B-Moses,B-Norm,B-CC,Rouge-1,Rouge-2,Rouge-L,Meteor
"(cpp,)",5.63,11.22,6.55,15.83,6.71,15.46,12.59
"(csharp,)",3.41,9.35,4.83,13.26,5.45,13.01,10.08
"(java,)",3.73,8.93,4.86,12.27,4.64,11.97,10.04
"(javascript,)",17.68,23.01,17.83,28.14,16.89,27.68,26.35
"(python,)",5.05,11.04,6.41,15.98,6.94,15.60,12.31


### MCMD<sub>2-PL</sub>

In [22]:
comb_finetune("javascript",2)

,B-Moses,B-Norm,B-CC,Rouge-1,Rouge-2,Rouge-L,Meteor
"(cpp, csharp)",5.89,11.51,7.00,15.94,7.14,15.58,13.01
"(cpp, java)",6.32,11.90,7.25,16.44,7.50,16.06,13.46
"(cpp, javascript)",12.57,17.38,13.20,21.61,12.93,21.22,18.86
"(cpp, python)",6.06,11.58,7.25,16.31,7.49,15.92,12.68
"(csharp, java)",4.59,10.28,5.81,14.31,6.02,13.95,11.43
"(csharp, javascript)",17.87,23.12,18.00,28.18,17.07,27.69,26.40
"(csharp, python)",5.24,11.22,6.55,16.13,7.01,15.70,12.51
"(java, javascript)",17.77,22.98,17.89,28.05,16.96,27.56,26.31
"(java, python)",5.58,11.25,6.78,15.94,6.92,15.49,12.77
"(javascript, python)",17.89,23.13,17.99,28.28,17.08,27.77,26.51


### MCMD<sub>3-PL</sub>

In [23]:
comb_finetune("javascript",3)

,B-Moses,B-Norm,B-CC,Rouge-1,Rouge-2,Rouge-L,Meteor
"(cpp, csharp, java)",6.22,11.91,7.19,16.51,7.51,16.10,13.35
"(cpp, csharp, javascript)",13.47,18.83,14.11,23.56,13.98,23.12,20.26
"(cpp, csharp, python)",6.32,12.03,7.59,16.36,7.65,15.94,13.06
"(cpp, java, javascript)",18.06,23.30,18.15,28.54,17.27,28.02,26.72
"(cpp, java, python)",6.83,12.53,7.89,17.61,7.98,17.12,14.48
"(cpp, javascript, python)",12.10,17.41,12.88,21.71,12.62,21.30,18.50
"(csharp, java, javascript)",18.06,23.20,18.13,28.40,17.20,27.89,26.65
"(csharp, java, python)",5.38,11.36,6.54,16.34,6.97,15.86,12.67
"(csharp, javascript, python)",17.96,23.31,18.10,28.48,17.19,27.96,26.66
"(java, javascript, python)",17.86,23.17,18.03,28.33,17.18,27.82,26.53


### MCMD<sub>4-PL</sub>

In [24]:
comb_finetune("javascript",4)

,B-Moses,B-Norm,B-CC,Rouge-1,Rouge-2,Rouge-L,Meteor
"(cpp, csharp, java, javascript)",18.07,23.15,18.10,28.34,17.27,27.84,26.53
"(cpp, csharp, java, python)",6.14,12.05,7.47,16.45,7.63,15.99,12.96
"(cpp, csharp, javascript, python)",18.02,23.19,18.11,28.41,17.19,27.88,26.62
"(cpp, java, javascript, python)",18.02,23.28,18.09,28.57,17.21,28.04,26.69
"(csharp, java, javascript, python)",18.17,23.39,18.23,28.59,17.33,28.08,26.79


### MCMD<sub>5-PL</sub>

In [25]:
comb_finetune("javascript",5)

,B-Moses,B-Norm,B-CC,Rouge-1,Rouge-2,Rouge-L,Meteor
"(cpp, csharp, java, javascript, python)",18.13,23.37,18.26,28.62,17.37,28.07,26.82


## Test PL: C++

### MCMD<sub>1-PL</sub>

In [13]:
comb_finetune("cpp",1) 

,B-Moses,B-Norm,B-CC,Rouge-1,Rouge-2,Rouge-L,Meteor
"(cpp,)",10.58,17.09,12.02,21.62,11.71,21.25,19.02
"(csharp,)",1.97,6.81,3.25,10.14,3.65,9.93,7.42
"(java,)",1.98,6.70,3.23,9.40,3.05,9.14,7.02
"(javascript,)",1.67,6.82,2.84,10.36,3.19,10.11,7.00
"(python,)",1.67,7.36,3.04,11.38,3.80,11.08,7.64


### MCMD<sub>2-PL</sub>

In [26]:
comb_finetune("cpp",2)

,B-Moses,B-Norm,B-CC,Rouge-1,Rouge-2,Rouge-L,Meteor
"(cpp, csharp)",10.70,17.02,12.00,21.45,11.70,21.07,19.00
"(cpp, java)",10.58,16.98,11.98,21.42,11.80,21.04,18.90
"(cpp, javascript)",7.31,12.57,8.23,16.15,8.74,15.82,12.72
"(cpp, python)",10.00,16.04,11.25,20.53,11.21,20.15,17.46
"(csharp, java)",2.16,7.49,3.42,11.20,3.97,10.88,7.83
"(csharp, javascript)",1.95,7.48,3.20,11.60,4.11,11.28,7.68
"(csharp, python)",2.02,7.69,3.35,11.76,4.22,11.45,8.14
"(java, javascript)",2.05,7.39,3.29,11.16,3.82,10.81,7.56
"(java, python)",1.97,7.71,3.23,11.27,3.78,10.96,7.77
"(javascript, python)",1.85,7.88,3.18,12.23,4.12,11.85,8.04


### MCMD<sub>3-PL</sub>

In [28]:
comb_finetune("cpp",3)

,B-Moses,B-Norm,B-CC,Rouge-1,Rouge-2,Rouge-L,Meteor
"(cpp, csharp, java)",10.84,17.09,12.17,21.59,11.86,21.21,19.19
"(cpp, csharp, javascript)",8.22,14.01,9.19,18.06,9.59,17.61,14.42
"(cpp, csharp, python)",8.49,14.47,9.47,18.59,9.62,18.14,15.13
"(cpp, java, javascript)",10.92,17.35,12.28,22.03,12.02,21.57,19.55
"(cpp, java, python)",10.98,17.34,12.35,22.04,12.10,21.61,19.63
"(cpp, javascript, python)",7.60,12.98,8.39,16.25,8.46,15.88,12.89
"(csharp, java, javascript)",2.17,7.91,3.45,12.08,4.27,11.67,8.12
"(csharp, java, python)",1.96,7.83,3.30,12.01,4.11,11.64,8.04
"(csharp, javascript, python)",2.22,8.17,3.51,12.41,4.41,12.02,8.44
"(java, javascript, python)",1.95,7.95,3.35,11.89,4.09,11.51,8.12


### MCMD<sub>4-PL</sub>

In [29]:
comb_finetune("cpp",4)

,B-Moses,B-Norm,B-CC,Rouge-1,Rouge-2,Rouge-L,Meteor
"(cpp, csharp, java, javascript)",11.08,17.23,12.39,21.85,12.09,21.39,19.50
"(cpp, csharp, java, python)",8.42,14.48,9.43,18.62,9.70,18.16,15.09
"(cpp, csharp, javascript, python)",11.04,17.52,12.38,22.22,12.10,21.73,19.65
"(cpp, java, javascript, python)",11.18,17.48,12.50,22.21,12.19,21.76,19.74
"(csharp, java, javascript, python)",2.35,8.26,3.66,12.67,4.46,12.23,8.72


### MCMD<sub>5-PL</sub>

In [30]:
comb_finetune("cpp",5)

,B-Moses,B-Norm,B-CC,Rouge-1,Rouge-2,Rouge-L,Meteor
"(cpp, csharp, java, javascript, python)",11.14,17.61,12.51,22.43,12.29,21.97,19.79


## Test PL: `C#`

### MCMD<sub>1-PL</sub>

In [14]:
comb_finetune("csharp",1)

,B-Moses,B-Norm,B-CC,Rouge-1,Rouge-2,Rouge-L,Meteor
"(cpp,)",1.94,7.26,3.39,9.17,3.39,9.02,7.47
"(csharp,)",24.25,24.03,20.28,27.31,19.13,27.03,26.01
"(java,)",1.60,6.11,2.92,7.28,2.32,7.16,6.37
"(javascript,)",1.48,6.52,2.72,8.91,2.57,8.76,6.93
"(python,)",1.65,6.85,2.92,9.24,3.20,9.08,7.42


### MCMD<sub>2-PL</sub>

In [31]:
comb_finetune("csharp",2)

,B-Moses,B-Norm,B-CC,Rouge-1,Rouge-2,Rouge-L,Meteor
"(cpp, csharp)",23.86,23.65,19.94,26.85,18.79,26.56,25.68
"(cpp, java)",2.49,7.48,3.80,9.51,3.68,9.35,7.98
"(cpp, javascript)",2.73,7.37,3.96,9.00,3.72,8.84,7.59
"(cpp, python)",2.57,7.59,3.95,9.59,3.98,9.42,7.80
"(csharp, java)",24.03,23.97,20.12,27.21,18.94,26.91,25.83
"(csharp, javascript)",24.28,24.32,20.41,27.67,19.24,27.35,26.28
"(csharp, python)",24.20,24.27,20.30,27.50,19.08,27.20,26.21
"(java, javascript)",2.06,6.87,3.24,8.88,3.05,8.72,7.25
"(java, python)",2.12,6.87,3.33,8.93,3.29,8.76,7.35
"(javascript, python)",2.18,7.97,3.49,10.71,3.79,10.51,8.32


### MCMD<sub>3-PL</sub>

In [32]:
comb_finetune("csharp",3)

,B-Moses,B-Norm,B-CC,Rouge-1,Rouge-2,Rouge-L,Meteor
"(cpp, csharp, java)",23.99,23.87,20.05,27.10,18.86,26.79,25.88
"(cpp, csharp, javascript)",21.03,20.59,17.09,23.71,16.51,23.44,21.46
"(cpp, csharp, python)",21.42,21.24,17.56,24.45,16.91,24.15,22.27
"(cpp, java, javascript)",2.66,7.90,3.89,10.16,3.75,9.95,8.43
"(cpp, java, python)",2.89,8.05,4.13,10.43,4.20,10.25,8.69
"(cpp, javascript, python)",3.11,8.36,4.58,9.66,4.10,9.49,8.14
"(csharp, java, javascript)",24.33,24.31,20.38,27.58,19.17,27.26,26.25
"(csharp, java, python)",23.78,23.84,19.91,27.13,18.81,26.82,25.77
"(csharp, javascript, python)",24.41,24.42,20.47,27.70,19.27,27.38,26.42
"(java, javascript, python)",2.08,7.39,3.39,9.76,3.57,9.59,7.66


### MCMD<sub>4-PL</sub>

In [33]:
comb_finetune("csharp",4)

,B-Moses,B-Norm,B-CC,Rouge-1,Rouge-2,Rouge-L,Meteor
"(cpp, csharp, java, javascript)",24.13,23.95,20.19,27.22,18.98,26.90,25.95
"(cpp, csharp, java, python)",21.60,21.12,17.55,24.50,17.00,24.19,22.44
"(cpp, csharp, javascript, python)",24.03,24.10,20.14,27.46,19.03,27.14,26.08
"(cpp, java, javascript, python)",3.03,8.23,4.24,10.70,4.15,10.48,8.83
"(csharp, java, javascript, python)",24.09,24.15,20.22,27.48,19.06,27.16,26.09


### MCMD<sub>5-PL</sub>

In [34]:
comb_finetune("csharp",5)

,B-Moses,B-Norm,B-CC,Rouge-1,Rouge-2,Rouge-L,Meteor
"(cpp, csharp, java, javascript, python)",24.19,24.29,20.33,27.70,19.20,27.36,26.27
